<a href="https://colab.research.google.com/github/kdhol/HiteshChaudhary_openSorceCont/blob/main/AI_Influencer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15095 MiB


In [7]:
### Dependency installations
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!sudo apt install python3.8

!sudo apt-get install python3.8-distutils

!python --version

!apt-get update

!apt install software-properties-common

!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel

!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/Winfredy/SadTalker &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt
!pip install openai
!pip install gtts
!pip install diffusers accelerate transformers
!pip install requests

update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,076 kB of archives.
After this operation, 18.8 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8-minimal amd64 3.8.20-1+jammy1 [796 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.8-minimal amd64 3.8.20-1+jammy1 [2,023 kB]
Get:3 https://ppa.launchpadcontent.net

In [6]:
### Models used for lisyncing
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

Download pre-trained models...
bash: scripts/download_models.sh: No such file or directory


In [ ]:
import requests
import uuid
import time
from openai import OpenAI
import json
from gtts import gTTS
import os

client = OpenAI(api_key="openai-key")

def get_prompt_for_image(characterstics):
    """Function to get the prompt for an Image.

    parameters:
    characterstics (str): Characterstics of the person for the image.

    return:
    str: Prompt for the image of the person.
    """

    prompt = f"""Given below are some characterstics of a person for a single scene in a video, give output a prompt for image generating model to generate a image of the person based on the given characerstics.

    Below are some guidelines for the image:
    A realistic human face, taken from the front.The person should look directly at the camera.Maintain a relaxed expression with a neutral or natural smile.Center the head and shoulders in the frame.Use a simple background with natural or soft lighting.Avoid the following: Cartoonish or non-human elements.Facing the camera sideways or positioning too close/far.Overexposed, underexposed, or harsh shadows.Moving objects such as animals or people in the background.

    Strictly give output in json format as below

    Output:
    {{"prompt": output_prompt}}

    Characterstics:
    {characterstics}

"""

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        temperature=0.5,
    )
    data = chat_completion.choices[0].message.content
    try:
        data = json.loads(data)
    except Exception as e:
        print("Error get youtube metadata", e, data)
        data = data.split('```json')[1].split('```')[0].replace('\n', '')
        data = json.loads(data)

    return data["prompt"]

In [ ]:
from diffusers import DiffusionPipeline

def generate_avatar_image(image_prompt):
  """Function to generate avatar image

  parameters:
  image_prompt (str): prompt to generate avatar image

  returns:
  image_path (str): path to the generated avatar image
  """

  pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1").to("cuda")
  image = pipe(image_prompt).images[0]
  print(image)
  image_path = "examples/source_image/generated_image.png"
  image.save(image_path)

  return image_path

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from gtts import gTTS
import os

def generate_voiceover(text, filename):
    """Function to generate the voiceover using gTTS

    parameters:
    text (str): the script text for which voiceover needs to be generated
    filename (str): name of the file with which voiceover will be saved

    """

    tts = gTTS(text)
    tts.save("temp.mp3")
    os.system(f"ffmpeg -i temp.mp3 -ar 16000 -ac 1 {filename}")
    os.remove("temp.mp3")



In [ ]:
# Import necessary libraries
from IPython.display import HTML  # Used for displaying HTML content in Jupyter notebooks
from base64 import b64encode  # Used for encoding binary data to base64 format
import os, sys  # OS for file handling, sys for system-related operations
import glob  # Used for pattern-based file searches

def create_ai_influencer(image_path, audio_path):
    """
    Function to create an AI-generated influencer video using an image and a voiceover.

    Parameters:
    image_path (str): Path to the image file (input image for animation)
    audio_path (str): Path to the audio file (input voiceover for lip-syncing)

    Returns:
    results (list): List of generated video files in the './results/' directory
    """

    # Run a command-line script 'inference.py' using Python 3.8
    # This script generates an animated video from the provided image and audio
    # Arguments:
    # --driven_audio: The audio file for lip-syncing
    # --source_image: The input image file
    # --result_dir: Directory where output files will be stored
    # --still: Likely forces the animation to use a single still image
    # --preprocess full: Preprocesses the input image before animation
    # --enhancer gfpgan: Uses GFPGAN for face enhancement in the animation
    #!python3.8 inference.py --driven_audio {audio_path} --source_image {image_path} --result_dir ./results --still --preprocess full --enhancer gfpgan
    !python3.8 inference.py --source_image {image_path} --result_dir ./results --still --preprocess full --enhancer gfpgan

    # Get a sorted list of all generated files in the './results/' directory
    results = sorted(os.listdir('./results/'))

    # Return the list of generated output files
    return results





In [9]:
# Import necessary libraries
from IPython.display import HTML  # Used for displaying HTML content in Jupyter notebooks
from base64 import b64encode  # Used for encoding binary data to base64 format
import os, sys  # OS for file handling, sys for system-related operations
import glob  # Used for pattern-based file searches

def create_ai_influencer(image_path, audio_path):
    """
    Function to create an AI-generated influencer video using an image and a voiceover.

    Parameters:
    image_path (str): Path to the image file (input image for animation)
    audio_path (str): Path to the audio file (input voiceover for lip-syncing)

    Returns:
    results (list): List of generated video files in the './results/' directory
    """

    # Run a command-line script 'inference.py' using Python 3.8
    # This script generates an animated video from the provided image and audio
    # Arguments:
    # --driven_audio: The audio file for lip-syncing
    # --source_image: The input image file
    # --result_dir: Directory where output files will be stored
    # --still: Likely forces the animation to use a single still image
    # --preprocess full: Preprocesses the input image before animation
    # --enhancer gfpgan: Uses GFPGAN for face enhancement in the animation
    #!python3.8 inference.py --driven_audio {audio_path} --source_image {image_path} --result_dir ./results --still --preprocess full --enhancer gfpgan
    #!python3.8 inference.py --source_image {image_path} --result_dir ./results --still --preprocess full --enhancer gfpgan
    !python3.8 inference.py --source_image american.jpg --result_dir ./results --still --preprocess full --enhancer gfpgan

    # Get a sorted list of all generated files in the './results/' directory
    results = ''#sorted(os.listdir('./results/'))

    # Return the list of generated output files
    return results


def init(avatar_details, script):

    """Function to generate ai influencer videos by first generating the avatar image using
    the prompt and the characterstics and then generating the voiceover using script.
    With these generated image and voiceover, AI influencer video is generated.

    parameters:
    avatar_details (dict) : characterstics of the avatar
    script (str) : script to generate the voiceover

    returns:
    res (list) : list of the generated files in the folder
    """
    # characterstics = avatar_details.get('characterstics')
    # prompt = get_prompt_for_image(characterstics)
    # print("Image prompt: " + prompt)
    # avatar_image_url = generate_avatar_image(prompt)
    # print("avatar_image_url: " + avatar_image_url)
    # voice_path = "examples/driven_audio/audio.wav"
    # generate_voiceover(script,voice_path)

    res = create_ai_influencer('1 american.jpg','')
    print(res)


### Init function to generate the Influencer video
init({'characterstics':'male European, wearing glasses and leather jacket with a smile on face'}, "Our planet is a masterpiece of life, a delicate balance of ecosystems. But this harmony is under threat.Every year, 8 million tons of plastic enter our oceans, choking marine life. Forests, the lungs of our planet, are disappearing at an alarming rate. Climate change fuels disasters, from rising seas to devastating wildfires.But there is hope. Across the globe, people are rising to the challenge. Simple actions—reducing waste, conserving energy, and protecting wildlife—can make a difference.The future of our planet is in our hands. Together, we can turn the tide and protect our only home. Act now, because every choice matters.")


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth" to /content/SadTalker/gfpgan/weights/alignment_WFLW_4HG.pth

100% 185M/185M [00:01<00:00, 146MB/s]
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/SadTalker/gfpgan/weights/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 192MB/s] 
Traceback (most recent call last):
  File "inference.py", line 144, in <module>
    main(args)
  File "inference.py", line 36, in main
    preprocess_model = CropAndExtract(sadtalker_paths, device)
  File "/content/SadTalker/src/utils/preprocess.py", line 56, in __init__
    checkpoint = torch.load(sadtalker_path['path_of_net_recon_model'], map_location=torch.device(device))    
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 699, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py